In [248]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter import filedialog as fd
from PIL import ImageTk,Image  
import PIL

from collections import Counter
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
import math
import struct
import os
import heapq
from scipy import signal as sig

In [5]:
### Reading and Writing PGM Files (Greyscale Images) ###

def readpgm2(name):
# Reads a pgm file (ASCII/P2) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P2' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readpgm5(name):
# Reads a pgm file (binary/P5)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P5\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readpgm(name):
# Reads a pgm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    try:
        try:
            return readpgm5(name)
        except:
            return readpgm2(name)
    except:
        print("Error reading " + name)
        return

def writepgm(file_name, data):
# Writes a pgm file (binary/P5) given pgm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P5\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [36]:
### Reading and Writing PPM Files (Full-Color Images) ###

def readppm3(name):
# Reads a ppm file (ASCII/P3) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P3' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readppm6(name):
# Reads a pgm file (binary/P6)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P6\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(3*w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readppm(name):
# Reads a ppm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    try:
        try:
            return readppm6(name)
        except:
            return readppm3(name)
    except:
        print("Error reading " + name)
        return
    
def writeppm(file_name, data):
# Writes a ppm file (binary/P6) given ppm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P6\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], 3*w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [267]:
# Utility Functions

def convertHSV(r, g, b):
# Converts a color from the RGB color model to the HSV color model
# r, g, b values should be in range [0,255]
# Returns a triple of floats in range [0,1] corresponding to (h,s,v)

    rr = r/255
    gg = g/255
    bb = b/255
    maxx = max(rr,gg,bb)
    minn = min(rr,gg,bb)
    delt = maxx-minn
    
    # hue 
    if delt == 0:
        h = 0
    elif maxx == rr:
        h = (((gg-bb)/delt) % 6) / 6
    elif maxx == gg:
        h = (((bb-rr)/delt) + 2) / 6
    else: # maxx == bb
        h = (((rr-gg)/delt) + 4) / 6

    # saturation
    if maxx == 0:
        s = 0
    else:
        s = delt/maxx
        
    # value
    v = maxx
    
    return (h,s,v)

def convertRGB(h, s, v):
# Converts a color from the HSV color model to the RGB color model
# h, s, v values should be in range [0,1]
# Returns a triple of ints in range [0,255] ; (r,g,b)

    H = h*360
    c = v*s
    abss = ((H/60) % 2) - 1
    x = c * (1 - abs(abss))
    m = v - c
    
    if H < 60:
        (rr,gg,bb) = (c,x,0)
    elif H < 120:
        (rr,gg,bb) = (x,c,0)
    elif H < 180:
        (rr,gg,bb) = (0,c,x)
    elif H < 240:
        (rr,gg,bb) = (0,x,c)
    elif H < 300:
        (rr,gg,bb) = (x,0,c)
    else: # H < 360:
        (rr,gg,bb) = (c,0,x)
        
    r = round((rr+m) * 255)
    g = round((gg+m) * 255)
    b = round((bb+m) * 255)
    
    return (r,g,b)

def padded_data(data, d):
# Pads a matrix with zeros and returns a new 2d array
# d = number zeros before and after each row/col
# data = pgm data including header metadata

    padded = []
    dimX = data[0][0]
    #create row of zeros
    z_row = [0] * (d + dimX + d)
    
    for _ in range(d):
        padded.append(z_row)
    
    for r in data[2:]:
        row = []
        for _ in range(d):
            row.append(0)
        for c in r:
            row.append(c)
        for _ in range(d):
            row.append(0)
        padded.append(row)
        
    for _ in range(d):
        padded.append(z_row)
        
    return padded

def find_neighbors(p, x, y, n):
# Returns the values of the neighbors of a pixel
# p = padded data
# x,y is the pixel whose neighbors are being calculated
# n is the dimension of the square neighborhood
    #print("(x,y) = ("+str(x)+", "+str(y)+")")
    assert n%2 == 1 and n >= 3
    neighbors = []
    dist = int((n-1)/2)
    for i in range(x-dist, x+dist+1): #range(0, )
        for j in range(y-dist, y+dist+1):
            neighbors.append(p[i][j])
    return neighbors

def min_filt(data, n): 
# Applies a minimum filter onto the image
# data = pgm data including header metadata
# Filter size is nxn
# Returns data of the new greyscale image including metadata

    assert n%2 == 1 and n >= 3
    d = int((n-1)/2)
    p = padded_data(data, d)
    
    new_data = []
    new_data.append(data[0])
    new_data.append(data[1])
    for i,r in enumerate(p[d:(0-d)]):
        row = []
        for j,c in enumerate(r[d:(0-d)]):
            neighbors = find_neighbors(p, i+d, j+d, n)
            minn = min(neighbors)
            row.append(minn)
        new_data.append(row)
    return new_data

def scale_data(data):
# Scales data to the range [0,L-1] where L = number of gray levels
# data = pgm data including header metadata
# Returns data of the scaled image including metadata
    gray = data[1][0]
    minv = min([min(d) for d in data[2:]])
    shft = []
    for r in data[2:]:
        row = []
        for c in r:
            row.append(c-minv)
        shft.append(row)
    maxv = max([max(d) for d in shft])
    fact = gray/maxv
    scale = []
    scale.append(data[0])
    scale.append(data[1])
    for r in shft:
        row = []
        for c in r:
            row.append(int(c*fact))
        scale.append(row)
    return scale



In [313]:
def deep_points(dmap, p):
# Finds the indices of the deepest values in a depth map
# Deepest points determined by lowest values
# dmap: a 2D array representing the depth pgm of an image, incl. header metadata
# p: a decimal indicating which top percentile of points to return
# e.g. p=0.1 returns the top 0.1% deepest points
# Returns a list of (value, (x,y))

    assert p <= 100
    w = dmap[0][0]
    h = dmap[0][1]
    
    n = w*h # total number of pixels
    pp = int(n * p / 100) # number of pixels to return
    
    # heapq is by default a min heap
    # since we need a max heap, we negate c upon pushing
    heap = [] 
    for i,r in enumerate(dmap[2:]):
        for j,c in enumerate(r):
            if(len(heap) <= pp):
                heapq.heappush(heap, (-c,(i,j)))
            else:
                if -c > heap[0][0] and c != 0:
                    heapq.heapreplace(heap, (-c,(i,j)))
                    
    # again, we use nlargest to find the lowest values 
    # since our values are negative
    points = []
    for (_,point) in heapq.nlargest(pp, heap):
        points.append(point)
        
    return points
    
def est_atm_light(dmap, data):
# Given an image and its depth map, finds the value of atmospheric light
# dmap: a 2D array representing the depth pgm of an image, incl. header metadata
# data: a 2D array representing the ppm of an image, incl. header metadata

    deep =  deep_points(dmap, .1)
    A = 0
    for (x,y) in deep:
        r = data[x+2][3*y]
        g = data[x+2][(3*y)+1]
        b = data[x+2][(3*y)+2]
        (_,_,v) = convertHSV(r,g,b)
        if v>A: 
            A=v
    return A

def raw_depth_map(data):
# Finds the raw depth map of the given image
# data: a 2D array representing the ppm of an image, incl. header metadata
# Returns a 2D array representing the raw depth pgm of an image, incl. header metadata

    w = data[0][0]
    h = data[0][1]
    lvls = data[1][0]

    dmap = []
    dmap.append(data[0])
    dmap.append(data[1])
    
    # linear coeffs from study
    th0 = 0.121779
    th1 = 0.959710
    th2 = -0.780245
    sigma = 0.041337
    
    # random normal distribution
    eps = (np.random.normal(0, sigma, size=(h,w))).tolist()
    
    for r in range(h):
        row = []
        for c in range(w):
            rr = data[r+2][3*c]
            gg = data[r+2][(3*c)+1]
            bb = data[r+2][(3*c)+2]
            (_,s,v) = convertHSV(rr,gg,bb)
            # calculate depth
            d = th0 + (th1 * v) + (th2 * s) + eps[r][c]
            d = int(lvls*d)
            row.append(d)
        dmap.append(row)
    scaled = scale_data(dmap)
    return scale_data(dmap)

def guided_filt(guide, img, r, eps):
# Applies a guided image filter onto the image
# img, guide = pgm data including header metadata
# Returns data of the new greyscale image including metadata

# Code taken from https://github.com/jacob5412
# Algorithm from "Guided Image Filtering" by Kaiming He et al

    w = img[0][0]
    h = img[0][1]
    I = np.array(guide[2:])
    P = np.array(img[2:])
    window = np.ones((r, r)) / (r * r)

    meanI = sig.convolve2d(I, window, mode="same")
    meanP = sig.convolve2d(P, window, mode="same")

    corrI = sig.convolve2d(I * I, window, mode="same")
    corrIP = sig.convolve2d(I * P, window, mode="same")

    varI = corrI - meanI * meanI
    covIP = corrIP - meanI * meanP
    a = covIP / (varI + e)
    b = meanP - a * meanI

    meana = sig.convolve2d(a, window, mode="same")
    meanb = sig.convolve2d(b, window, mode="same")

    q = meana * I + meanb
    
    Q = q.astype(int).tolist()
    
    # add metadata 
    Q.insert(0, img[1])
    Q.insert(0, img[0])
    return Q

def final_depth_map(data):
# Finds the restored depth map of the given image
# data: a 2D array representing the ppm of an image, incl. header metadata
# Returns a 2D array representing the final, restored depth pgm of an image, incl. header metadata
 
    # filter size for min filter:
    n = 15 
    # filter size for guided filter:
    r = 8
    # regularization for edge detection
    eps = 0.2 * 0.2
    
    # raw depth map
    raw = raw_depth_map(data)
    # apply min/guided filters
    minn = min_filt(raw, n)
    gf = guided_filt(raw, minn, r, eps)
    # scale
    final = scale_data(gf)
    return final

In [314]:
dolls = readppm("images/dolls.ppm")
dolls_depth = readpgm("images/dolls_disp.pgm")

In [315]:
fdm = final_depth_map(dolls)
writepgm("fdm.pgm", fdm)